In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt

from warnings import simplefilter 
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
simplefilter(action="ignore", category=pd.errors.SettingWithCopyWarning)

In [2]:
####################################################
############# Setear segun cada maquina ############
#os.chdir("C:/Users/herna/labo3_empresa3_repo/")
#os.chdir("C:/diego_tools/labo3/")
os.chdir("/home/dcastro_contacto/buckets/b1/")
####################################################

In [3]:
#################################
###### Variables Generales ######
num_lags = 3
cant_clusters = 5

In [4]:
arch_cluster_prefijo = "datasets/emp3_sellout_base_cluster_"
arch_salida_prefijo = "datasets/emp3_sellout_train_cluster_"

In [5]:
def crear_nuevos_features(df_param):
    # Diferencia entre lo solicitado y lo entregado
    df_param["dif_cust_request_tn"]=df_param.cust_request_tn-df_param.tn
    df_param["dif_cust_request_tn_porc"]=np.where(df_param.cust_request_tn==0,0,100*df_param.dif_cust_request_tn/df_param.cust_request_tn)
   
    return df_param

In [6]:
def crear_lags_prod_cust(df_param):
    # Lags para product_id-customer_id
    for i in range(1, num_lags + 1):
        df_param[f'tn_lag_{i}'] = df_param.groupby(['product_id','customer_id'])['tn'].shift(i)

    return df_param

In [7]:
def crear_variable_target(df_param):
    df_param["tn_mas_2"] = df_param.groupby(['product_id','customer_id'])['tn'].shift(-2)

    return df_param

In [8]:
def crear_delta_lags_prod_cust(df_param):
    for i in range(1, num_lags+1):
        df_param[f'tn_delta_{i}'] = df_param.groupby(['product_id','customer_id'])['tn'].diff(i)

    return df_param

In [9]:
for cluster_iter in range(0,cant_clusters+1):
    df = pd.read_csv(arch_cluster_prefijo + str(cluster_iter) + ".csv")

    #Antes que nada, ordeno por prod-cust
    
    #Nuevos features
    df = crear_nuevos_features(df)

    # Lags
    df = crear_lags_prod_cust(df)

    # Variable Target
    df = crear_variable_target(df)

    df = crear_delta_lags_prod_cust(df)

    print(len(df))
    
    df.to_csv(arch_salida_prefijo + str(cluster_iter) + ".csv", index=False)

4020
3921692
3925039
3935373
3927537
3925446


## Verificaciones

In [10]:
# Nuevos features
df[["product_id","customer_id","periodo","tn","cust_request_tn","dif_cust_request_tn","dif_cust_request_tn_porc"]].sample(10)

,product_id,customer_id,periodo,tn,cust_request_tn,dif_cust_request_tn,dif_cust_request_tn_porc
3582567,21156,10421,201710,0.00000,0.00000,0.0,0.0
1804238,20502,10464,201804,0.00000,0.00000,0.0,0.0
2922147,20876,10085,201905,0.00000,0.00000,0.0,0.0
393096,20103,10543,201902,0.00000,0.00000,0.0,0.0
3276613,21016,10189,201708,0.00109,0.00109,0.0,0.0
153153,20040,10413,201807,0.00000,0.00000,0.0,0.0
348483,20092,10056,201810,0.00000,0.00000,0.0,0.0
2164600,20620,10461,201910,0.00000,0.00000,0.0,0.0
3457148,21084,10482,201901,0.00000,0.00000,0.0,0.0
2028079,20579,10244,201701,0.00000,0.00000,0.0,0.0


In [11]:
# Lags prod-cust
df[["product_id","customer_id","periodo","tn","tn_lag_1","tn_lag_2","tn_lag_3"]].head(50)

,product_id,customer_id,periodo,tn,tn_lag_1,tn_lag_2,tn_lag_3
0,20001,10014,201701,0.12312,NaN,NaN,NaN
1,20001,10014,201702,1.35435,0.12312,NaN,NaN
2,20001,10014,201703,12.07725,1.35435,0.12312,NaN
3,20001,10014,201704,0.61562,12.07725,1.35435,0.12312
4,20001,10014,201705,0.86186,0.61562,12.07725,1.35435
5,20001,10014,201706,0.00000,0.86186,0.61562,12.07725
6,20001,10014,201707,6.77177,0.00000,0.86186,0.61562
7,20001,10014,201708,11.97651,6.77177,0.00000,0.86186
8,20001,10014,201709,5.64127,11.97651,6.77177,0.00000
9,20001,10014,201710,1.23123,5.64127,11.97651,6.77177


In [12]:
# Variable Target
df[["product_id","customer_id","periodo","tn","tn_mas_2"]].head(50)

,product_id,customer_id,periodo,tn,tn_mas_2
0,20001,10014,201701,0.12312,12.07725
1,20001,10014,201702,1.35435,0.61562
2,20001,10014,201703,12.07725,0.86186
3,20001,10014,201704,0.61562,0.00000
4,20001,10014,201705,0.86186,6.77177
5,20001,10014,201706,0.00000,11.97651
6,20001,10014,201707,6.77177,5.64127
7,20001,10014,201708,11.97651,1.23123
8,20001,10014,201709,5.64127,4.29811
9,20001,10014,201710,1.23123,0.86186


In [13]:
# Delta Lags prod-cust
df[["product_id","customer_id","periodo","tn","tn_lag_1","tn_delta_1","tn_lag_2","tn_delta_2","tn_lag_3","tn_delta_3"]].head(50)

,product_id,customer_id,periodo,tn,tn_lag_1,tn_delta_1,tn_lag_2,tn_delta_2,tn_lag_3,tn_delta_3
0,20001,10014,201701,0.12312,NaN,NaN,NaN,NaN,NaN,NaN
1,20001,10014,201702,1.35435,0.12312,1.23123,NaN,NaN,NaN,NaN
2,20001,10014,201703,12.07725,1.35435,10.72290,0.12312,11.95413,NaN,NaN
3,20001,10014,201704,0.61562,12.07725,-11.46163,1.35435,-0.73873,0.12312,0.49250
4,20001,10014,201705,0.86186,0.61562,0.24624,12.07725,-11.21539,1.35435,-0.49249
5,20001,10014,201706,0.00000,0.86186,-0.86186,0.61562,-0.61562,12.07725,-12.07725
6,20001,10014,201707,6.77177,0.00000,6.77177,0.86186,5.90991,0.61562,6.15615
7,20001,10014,201708,11.97651,6.77177,5.20474,0.00000,11.97651,0.86186,11.11465
8,20001,10014,201709,5.64127,11.97651,-6.33524,6.77177,-1.13050,0.00000,5.64127
9,20001,10014,201710,1.23123,5.64127,-4.41004,11.97651,-10.74528,6.77177,-5.54054


######### Agregar cuando se pongan lags de prod, cust y cats

for i in range(1, num_lags + 1):
    df_cat1[f'cat1_tn_lag_{i}'] = df_cat1.groupby('cat1')['cat1_tn'].shift(i)
    df_cat1[f'cat1_tn_delta_{i}'] = df_cat1.groupby('cat1')['cat1_tn'].diff(i)

df_cat1.head()

df = df.merge(df_cat1,how="inner",on=["cat1","periodo"])
df.head()

df[["product_id","cat1","periodo","cat1_tn","cat1_tn_lag_1","cat1_tn_delta_1","cat1_tn_lag_2","cat1_tn_delta_2","cat1_tn_lag_3","cat1_tn_delta_3"]].sort_values(by=["product_id","periodo"],ascending=True).head(10)

df_cat1_hc = df_cat1[df_cat1.cat1=="HC"]


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3925446 entries, 0 to 3925445
Data columns (total 35 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   product_id                 int64  
 1   periodo                    int64  
 2   customer_id                int64  
 3   cero_ventas                int64  
 4   plan_precios_cuidados      float64
 5   cust_request_qty           float64
 6   cust_request_tn            float64
 7   tn                         float64
 8   periodo_fecha              object 
 9   mes                        int64  
 10  meses_historia_prod        int64  
 11  meses_historia_cliente     int64  
 12  cliente_estrella           int64  
 13  prod_cust                  object 
 14  cat1                       object 
 15  cat2                       object 
 16  cat3                       object 
 17  brand                      object 
 18  sku_size                   float64
 19  producto_estrella          float64
 20  te

In [15]:
df.isna().sum()

product_id                        0
periodo                           0
customer_id                       0
cero_ventas                       0
plan_precios_cuidados             0
cust_request_qty                  0
cust_request_tn                   0
tn                                0
periodo_fecha                     0
mes                               0
meses_historia_prod               0
meses_historia_cliente            0
cliente_estrella                  0
prod_cust                         0
cat1                              0
cat2                              0
cat3                              0
brand                             0
sku_size                          0
producto_estrella                 0
temp_media                        0
temp_max_media                    0
temp_min_media                    0
IPC                               0
promedio_mens_dolar_venta         0
catastrofe                        0
dif_cust_request_tn               0
dif_cust_request_tn_porc    

In [16]:
len(df.columns)

35